In [1]:
catchment_name = 'WykeBeck'

In [3]:
from shapely.geometry import box
import sys

sys.path.append("../ProcessModelResults/")
from my_functions import *

def save_clipped_to_gdf(raster, out_meta, catchment_gdf, fp):
    save_array_as_raster(raster, fp, out_meta)
    
    with rasterio.open(fp) as src:
        catchment_gdf=catchment_gdf.to_crs(src.crs)
        # print(Vector.crs)
        out_image, out_transform=mask(src,catchment_gdf.geometry,crop=True)
        out_meta=src.meta.copy() # copy the metadata of the source DEM

    out_meta.update({"driver":"Gtiff", "height":out_image.shape[1], # height starts with shape[1]
        "width":out_image.shape[2], # width starts with shape[2]
        "transform":out_transform
    })

    with rasterio.open(fp,'w',**out_meta) as dst:
        dst.write(out_image)  

In [5]:
model_directory = '../../../FloodModelling/{}Models/'.format(catchment_name)

In [8]:
# Specify strings relating to catchment
if catchment_name == 'LinDyke':
    minx, miny, maxx, maxy = 437000,  426500,  445500, 434300
    # catchment_gdf = gpd.read_file(model_directory + 'Model_IdealisedProfiles/CatchmentLinDyke_exported.shp')
    catchment_gdf = gpd.read_file(os.path.join(model_directory, 'SubCatchmentBoundaries/ManualTrimAboveWetlands.shp'))
    terrain_fp = model_directory + 'TerrainData/terrain.tif'
elif catchment_name == 'WykeBeck':
    minx, miny, maxx, maxy = 430004,  429978, 438660, 440996 
    catchment_gdf = gpd.read_file(model_directory + 'Model_IdealisedProfiles/Model_IdealisedProfiles/WykeBeckCatchment.shp')
    terrain_fp = model_directory + 'TerrainData/Terrain.wykeDEM.tif'
# Create a bounding box (this is used in preparing the rasters)
bbox = box(minx, miny, maxx, maxy)    

DriverError: ../../../FloodModelling/WykeBeckModels/Model_IdealisedProfiles/Model_IdealisedProfiles/WykeBeckCatchment.shp: No such file or directory

In [6]:
terrain, out_meta = open_and_clip(terrain_fp, bbox)

In [6]:
if catchment_name == 'LinDyke':
    original_landcover_fp = "LandCover_resized"
elif catchment_name == 'WykeBeck':
    original_landcover_fp = "LandCover_resized"
    original_landcover_fp = "LandCover_clipped_MA_Download_resampled2m"

landcover_clipped, out_meta = open_and_clip(model_directory + 'LandCoverData/{}.tif'.format(original_landcover_fp), bbox)

NameError: name 'bbox' is not defined

In [9]:
output_fp_terrain = model_directory + 'TerrainData/Terrain_clipped_withoutwetlands.tif'
save_clipped_to_gdf(terrain, out_meta, catchment_gdf, output_fp_terrain)

output_fp_landcover = model_directory + 'LandCoverData/LandCover_clipped_withoutwetlands.tif'
save_clipped_to_gdf(landcover_clipped, out_meta, catchment_gdf, output_fp_landcover)